**Setup the embeddings model**

In [ ]:
model_name = "Alibaba-NLP/gte-large-en-v1.5"
model_kwargs = {'device': 'cpu', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


**Load chromdb object from saved chromadb vector db**

In [ ]:
persist_directory = 'fulldb_cliuserguide'
!unzip "/content/drive/MyDrive/Colab Notebooks/My NN stuff/alteon/chroma_db_cliuserguide.zip"
vectordb_cliguide = Chroma(persist_directory=persist_directory, embedding_function=hf)
print(vectordb_cliguide._collection.count())

**Convert to pandas df**

In [ ]:
top = vectordb_cliguide.get(include=['documents','metadatas', 'embeddings'])
newdict = {}
newdict['ids'] = top['ids']
newdict['metadatas'] = top['metadatas']
newdict['docs'] = top['documents']
newdict['embedding']= top['embeddings']

df = pd.DataFrame(newdict)
df = df.rename(columns={"docs":"name"}, errors="raise")

# customize id per your requirements
df['id'] = ["cliguide_chunks/"+str(i) for i in range(1, len(df) + 1)]

# verify
print (df.head(3))
print (df.tail(3))


############## convert to json file ###########


**Convert pandas to json and write to disk**

In [ ]:
jsonstring = df[["id","name","embedding"]].to_json(orient="records", lines=True)
with open("cliguideembeddings.json", "w") as f:
    f.write(jsonstring)